In [103]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch
import numpy as np

set_seed(43)

In [2]:
pip install transformers torch

In [3]:
model_name = "EleutherAI/pythia-2.8b"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
example_input_text = "The capital of France is"
# example_input_text = "What is the capital of France?"
example_inputs = tokenizer(example_input_text, return_tensors="pt")

In [5]:
input_ids = example_inputs['input_ids']
attention_mask = example_inputs['attention_mask']  # Add attention mask

In [53]:
outputs = model.generate(
    input_ids,
    attention_mask=attention_mask,            # Pass the attention mask
    max_length=50,                            # Maximum length of generated text
    num_return_sequences=1,                   # Number of outputs to generate
    temperature=1,                            # Balanced temperature to avoid extreme values
    #top_p=0.9,                                # Nucleus sampling parameter for diversity
    do_sample=False,                          # Enable sampling for non-deterministic outputs
    output_scores=True,                       # Return logits (scores) for each generated token
    output_logits=True,                        # Return logits (probs) for each generated token
    return_dict_in_generate=True,             # Return a dictionary with additional data
    pad_token_id=tokenizer.eos_token_id,      # Set pad_token_id to eos_token_id
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [81]:
# Extract generated output tokens
generated_ids = outputs['sequences'][len(example_input_text):]

# Decode the generated output to text
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Model's Output:", output_text[len(example_input_text):])

Model's Output: The capital of France is Paris, and the capital of the United States is Washington.

The capital of the United States is Washington, D.C.

The capital of France is Paris, and the capital of the United States is Washington


In [ ]:
logits = torch.stack(outputs.logits, dim=1)[0]
logits = logits.detach().double()

In [99]:
print("\nLogits for each generated word:")
for i, word_logits in enumerate(logits):
    token_id = generated_ids[0, len(input_ids[0]) + i].item()  # Start from the first generated token
    word = tokenizer.decode([token_id])
    print(f"Logits for token {i + 1} ({word}): {word_logits}")


Logits for each generated word:
Logits for token 1 ( Paris): tensor([ 5.3485, -3.6766,  5.3073,  ..., -3.5339, -3.4105, -3.8331],
       dtype=torch.float64)
Logits for token 2 (,): tensor([10.2937, -2.1314, 11.7194,  ..., -2.4259, -2.3769, -2.3177],
       dtype=torch.float64)
Logits for token 3 ( and): tensor([ 6.1700, -3.0431,  2.5693,  ..., -3.3752, -3.1665, -3.1515],
       dtype=torch.float64)
Logits for token 4 ( the): tensor([ 4.3976, -2.8417,  2.5150,  ..., -3.0532, -2.8512, -2.9520],
       dtype=torch.float64)
Logits for token 5 ( capital): tensor([ 3.9163, -3.9603,  2.1845,  ..., -4.3546, -4.0860, -4.1736],
       dtype=torch.float64)
Logits for token 6 ( of): tensor([ 8.3402, -0.6363,  5.8462,  ..., -1.0571, -0.9845, -1.0346],
       dtype=torch.float64)
Logits for token 7 ( the): tensor([ 4.1348, -4.2191,  1.8183,  ..., -4.0464, -3.9459, -4.1022],
       dtype=torch.float64)
Logits for token 8 ( United): tensor([ 5.3471, -4.2211,  1.2877,  ..., -4.2811, -4.1759, -4.4154]

In [107]:
for i, word_logits in enumerate(logits):
    # Find the word corresponding to the generated token
    token_id = generated_ids[0, len(input_ids[0]) + i].item()  # Start from the first generated token
    word = tokenizer.decode([token_id])

    # Find the logit for the predicted word
    predicted_word_logit = word_logits[token_id]

    # Find the maximum logit and the corresponding word
    max_logit =torch.max(word_logits)
    max_logit_token_id = torch.argmax(word_logits)
    max_logit_word = tokenizer.decode([max_logit_token_id])

    print(f"------ Token {i + 1} ------")
    print(f"Predicted word '{word}': Logit = {predicted_word_logit:.4f}")
    print(f"Max Logit = {max_logit:.4f}, Word with Max Logit = {max_logit_word}")
    print(f"Predicted word == Word with Max Logit:  {word == max_logit_word}")

------ Token 1 ------
Predicted word ' Paris': Logit = 14.5235
Max Logit = 14.5235, Word with Max Logit =  Paris
Predicted word == Word with Max Logit:  True
------ Token 2 ------
Predicted word ',': Logit = 16.8072
Max Logit = 16.8072, Word with Max Logit = ,
Predicted word == Word with Max Logit:  True
------ Token 3 ------
Predicted word ' and': Logit = 14.3390
Max Logit = 14.3390, Word with Max Logit =  and
Predicted word == Word with Max Logit:  True
------ Token 4 ------
Predicted word ' the': Logit = 14.8740
Max Logit = 14.8740, Word with Max Logit =  the
Predicted word == Word with Max Logit:  True
------ Token 5 ------
Predicted word ' capital': Logit = 13.5860
Max Logit = 13.5860, Word with Max Logit =  capital
Predicted word == Word with Max Logit:  True
------ Token 6 ------
Predicted word ' of': Logit = 19.5528
Max Logit = 19.5528, Word with Max Logit =  of
Predicted word == Word with Max Logit:  True
------ Token 7 ------
Predicted word ' the': Logit = 14.1146
Max Logit =

In [100]:
# test_logits = torch.stack(outputs['scores'], dim=1)  # Shape: (1, generated_sequence_length, vocab_size)

# # Convert logits to numpy for easier processing
# logits_np = test_logits[0].detach().cpu().numpy()

In [101]:
# print("\nLogits for each generated word:")
# for i, word_logits in enumerate(logits_np):
#     token_id = generated_ids[0, len(input_ids[0]) + i].item()  # Start from the first generated token
#     word = tokenizer.decode([token_id])
#     print(f"Logits for token {i + 1} ({word}): {word_logits}")


Logits for each generated word:
Logits for token 1 ( Paris): [ 5.3484616 -3.676563   5.307328  ... -3.5338824 -3.4105015 -3.833106 ]
Logits for token 2 (,): [10.2936535 -2.1314034 11.719395  ... -2.4259005 -2.3768988 -2.317661 ]
Logits for token 3 ( and): [ 6.1699634 -3.0431135  2.5692863 ... -3.3752065 -3.1664968 -3.1515303]
Logits for token 4 ( the): [ 4.3975983 -2.8416598  2.5150442 ... -3.0532064 -2.8512018 -2.9519892]
Logits for token 5 ( capital): [ 3.916289  -3.9602578  2.1844769 ... -4.3546157 -4.085994  -4.1736465]
Logits for token 6 ( of): [ 8.340183   -0.63627744  5.846176   ... -1.0571425  -0.98445654
 -1.0346277 ]
Logits for token 7 ( the): [ 4.134814  -4.2191343  1.8182817 ... -4.0463886 -3.9459407 -4.102175 ]
Logits for token 8 ( United): [ 5.347087  -4.2210774  1.287687  ... -4.281147  -4.175948  -4.4154243]
Logits for token 9 ( States): [ 9.133443  -4.2754784  3.7846336 ... -4.659781  -4.0405846 -4.4069176]
Logits for token 10 ( is): [ 9.512566  -2.0155013  6.9007416 

In [78]:
# for i, word_logits in enumerate(logits_np):
#     # Find the word corresponding to the generated token
#     token_id = generated_ids[0, len(input_ids[0]) + i].item()  # Start from the first generated token
#     word = tokenizer.decode([token_id])

#     # Find the logit for the predicted word
#     predicted_word_logit = word_logits[token_id]

#     # Find the maximum logit and the corresponding word
#     max_logit = np.max(word_logits)
#     max_logit_token_id = np.argmax(word_logits)
#     max_logit_word = tokenizer.decode([max_logit_token_id])

#     print(f"------ Token {i + 1} ------")
#     print(f"Predicted word '{word}': Logit = {predicted_word_logit:.4f}")
#     print(f"Max Logit = {max_logit:.4f}, Word with Max Logit = {max_logit_word}")
#     print(f"Predicted word == Word with Max Logit:  {word == max_logit_word}")

------ Token 1 ------
Predicted word ' Paris': Logit = 14.5235
Max Logit = 14.5235, Word with Max Logit =  Paris
Predicted word == Word with Max Logit:  True
------ Token 2 ------
Predicted word '.': Logit = 16.6619
Max Logit = 16.8072, Word with Max Logit = ,
Predicted word == Word with Max Logit:  False
------ Token 3 ------
Predicted word ' It': Logit = 13.1378
Max Logit = 13.8188, Word with Max Logit = 

Predicted word == Word with Max Logit:  False
------ Token 4 ------
Predicted word ' has': Logit = -inf
Max Logit = 13.5907, Word with Max Logit =  a
Predicted word == Word with Max Logit:  False
------ Token 5 ------
Predicted word ' many': Logit = 11.1955
Max Logit = 11.4185, Word with Max Logit =  beautiful
Predicted word == Word with Max Logit:  False
------ Token 6 ------
Predicted word ' museums': Logit = -inf
Max Logit = 17.8046, Word with Max Logit = -
Predicted word == Word with Max Logit:  False
------ Token 7 ------
Predicted word ',': Logit = -inf
Max Logit = 18.3732, W